In [1]:
import sys
import os
import matplotlib.pyplot as plt

# Add the src directory to the path. TEMPORARY FIX
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from models.unet.unet import UNet
from models.unet.unet_formes import UNetFormes
from src.data_processing.dataset_loader import CoastData


In [6]:
# Load the data to split it and save it to a dict
path = os.path.abspath(os.path.join(os.getcwd(), "../../data/processed/"))
data = CoastData(data_path=path)

data_split = data.split_data()

CoastData: global - 1717 images
Coast: agrelo, Total size: 244
Coast: arenaldentem, Total size: 40
Coast: cadiz, Total size: 946
Coast: cies, Total size: 430
Coast: samarador, Total size: 57


In [7]:
unet = UNet(num_classes=4)
# print(unet.model)

In [8]:
# Load the data to the model
data = unet.load_data(data_split, UNetFormes)

In [ ]:
# Train the model
unet.train(epochs=1)